# Lesson 1：   Data extraction fundamentals(数据提取基础)

## 简介

Data --> Gathering --> Extracting --> cleaning --> Storing --> Analysis 

We need to asses out data to
 - TEST ASSUMPTION ABOUT
  - VALUES
  - DATA TYPE
  - SHAPE
 - IDENTITFY ERRORS OR OUTLINERS(异常值)
 - FIND MISSING VALUES


> 数据科学家往往关心数据包含哪些项（what items），以及这些项有哪些字段

***常见数据格式***：
- 表格
- CSV （TSV是以制表符为分隔符）
- JSON
- HTML

In [1]:
# prictice --> Parsing CSV Files
import os
# -*- coding: UTF-8 -*-

DATADIR = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/"
DATAFILE = "beatles-diskography.csv"


def parse_file(datafile):
    data = []
    with open(datafile, "r") as f:       
        header = f.readline().split(',')
        count = 0
        for line in f:
            if count == 10:
                break
            
            fields = line.split(',')
            entry = {}
            
            for i,value in enumerate(fields):
                entry[header[i].strip()] = value.strip()
            
            count +=1
            data.append(entry)

    return data


def test():
    # a simple test of your implemetation
    datafile = os.path.join(DATADIR, DATAFILE)
    d = parse_file(datafile)
    
    firstline = {'Title': 'Please Please Me', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '22 March 1963', 'US Chart Position': '-', 'RIAA Certification': 'Platinum', 'BPI Certification': 'Gold'}
    tenthline = {'Title': '', 'UK Chart Position': '1', 'Label': 'Parlophone(UK)', 'Released': '10 July 1964', 'US Chart Position': '-', 'RIAA Certification': '', 'BPI Certification': 'Gold'}

    assert d[0] == firstline
    assert d[9] == tenthline
    
test()

### 9. Using CSV module

Python在处理CSV文件时，调用CSV模块，他能为你处理任何CSV能遇到的问题

In [2]:
import csv

def parse_csv(datafile):
    data = []
    with open(datafile,'rb') as sd:
        r = csv.DictReader(sd)
        
        for line in r:
            data.append(line)
    return data 


datafile = os.path.join(DATADIR, DATAFILE)           
parse_csv(datafile)[13]           

{'BPI Certification': '',
 'Label': 'Parlophone(NZ), Capitol(US)',
 'RIAA Certification': 'Platinum',
 'Released': '14 June 1965',
 'Title': 'Beatles VI',
 'UK Chart Position': '-',
 'US Chart Position': '1'}

### 10. Introduce to xlrd
***Help on package xlrd***:   
NAME  
* xlrd  

PACKAGE CONTENTS  
* biffh  
* book  
* compdoc  
* formatting  
* formula  
* info  
* licences  
* sheet
  * Cell    
  * Colinfo
  * Hyperlink
  * MSODrawing
  * MSObj
  * MSTxo
  * Note
  * Rowinfo
  * Sheet
    * cell(self, rowx, colx)
    * cell_type(self, rowx, colx)
    * cell_value(self, rowx, colx)
    * cell_xf_index(self, rowx, colx)
    * col = col_slice(self, colx, start_rowx=0, end_rowx=None)
    * col_slice(self, colx, start_rowx=0, end_rowx=None)
    * col_types(self, colx, start_rowx=0, end_rowx=None)
    * col_values(self, colx, start_rowx=0, end_rowx=None)
    * computed_column_width(self, colx)
    * row(self, rowx)
    * row_len(self, rowx)
    * row_slice(self, rowx, start_colx=0, end_colx=None)
    * row_types(self, rowx, start_colx=0, end_colx=None)
    * row_values(self, rowx, start_colx=0, end_colx=None)
* timemachine  
* xldate  
* xlsx  


In [6]:
import os
import xlrd

DATADIR = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/"
filename = "2013_ERCOT_Hourly_Load_Data.xls"
datafile = os.path.join(DATADIR,filename)

def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)

    data = [[sheet.cell_value(r, col) 
                for col in range(sheet.ncols)] 
                    for r in range(sheet.nrows)]

    print "\nList Comprehension"
    print "data[3][2]:",
    print data[3][2]

    print "\nCells in a nested loop:"    
    for row in range(sheet.nrows):
        for col in range(sheet.ncols):
            if row == 50:
                print sheet.cell_value(row, col),


    ### other useful methods:
    print "\nROWS, COLUMNS, and CELLS:"
    print "Number of rows in the sheet:", 
    print sheet.nrows
    print "Type of data in cell (row 3, col 2):", 
    print sheet.cell_type(3, 2)
    print "Value in cell (row 3, col 2):", 
    print sheet.cell_value(3, 2)
    print "Get a slice of values in column 3, from rows 1-3:"
    print sheet.col_values(3, start_rowx=1, end_rowx=4)

    print "\nDATES:"
    print "Type of data in cell (row 1, col 0):", 
    print sheet.cell_type(1, 0)
    exceltime = sheet.cell_value(1, 0)
    print "Time in Excel format:",
    print exceltime
    print "Convert time to a Python datetime tuple, from the Excel float:",
    print xlrd.xldate_as_tuple(exceltime, 0)

    return data

data = parse_file(datafile)


List Comprehension
data[3][2]: 1036.088697

Cells in a nested loop:
41277.0833333 9238.73731 1438.20528 1565.442856 916.708348 14010.903488 3027.98334 6165.211119 1157.741663 37520.933404 
ROWS, COLUMNS, and CELLS:
Number of rows in the sheet: 7296
Type of data in cell (row 3, col 2): 2
Value in cell (row 3, col 2): 1036.088697
Get a slice of values in column 3, from rows 1-3:
[1411.7505669999982, 1403.4722870000019, 1395.053150000001]

DATES:
Type of data in cell (row 1, col 0): 3
Time in Excel format: 41275.0416667
Convert time to a Python datetime tuple, from the Excel float: (2013, 1, 1, 1, 0, 0)


### 11. Quiz: Reading Excel File

主要是针对单元格，行，列的操作 练习

In [138]:

"""
Your task is as follows:
- read the provided Excel file
- find and return the min, max and average values for the COAST region
- find and return the time value for the min and max entries
- the time values should be returned as Python tuples

Please see the test function for the expected return format

"""

import os
import xlrd

DATADIR = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/"
filename = "2013_ERCOT_Hourly_Load_Data.xls"
datafile = os.path.join(DATADIR,filename)

def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    
    data = [[sheet.cell_value(r, col) 
                for col in range(sheet.ncols)] 
                    for r in range(sheet.nrows)]
    
    
    cv = sheet.col_values(1,start_rowx=1,end_rowx=None)
    
    maxvalue = max(cv)
    minvalue = min(cv)

    maxpos = cv.index(maxvalue) + 1
    minpos = cv.index(minvalue) + 1
    
    maxtime = sheet.cell_value(maxpos,0)
    max_real_time = xlrd.xldate_as_tuple(maxtime,0)
    mintime = sheet.cell_value(minpos,0)
    min_real_time = xlrd.xldate_as_tuple(mintime,0)
    
    col_len = float(len(cv))
    
    avgcoast = sum(cv)/col_len

    
    data = {
            'maxtime': max_real_time,
            'maxvalue': maxvalue,
            'mintime': min_real_time,
            'minvalue': minvalue,
            'avgcoast': avgcoast
    }
    return data  


def test():
    data = parse_file(datafile)

    assert data['maxtime'] == (2013, 8, 13, 17, 0, 0)
    assert round(data['maxvalue'], 10) == round(18779.02551, 10)


test()

### 15. Quiz: JSSON playgroud

In [145]:
"""
To experiment with this code freely you will have to run this code locally.
Take a look at the main() function for an example of how to use the code. We
have provided example json output in the other code editor tabs for you to look
at, but you will not be able to run any queries through our UI.
"""
import json
import requests

BASE_URL = "http://musicbrainz.org/ws/2/"
ARTIST_URL = BASE_URL + "artist/"


# query parameters are given to the requests.get function as a dictionary; this
# variable contains some starter parameters.
query_type = {  "simple": {},
                "atr": {"inc": "aliases+tags+ratings"},
                "aliases": {"inc": "aliases"},
                "releases": {"inc": "releases"}}


def query_site(url, params, uid="", fmt="json"):
    """
    This is the main function for making queries to the musicbrainz API. The
    query should return a json document.
    """
    params["fmt"] = fmt
    r = requests.get(url + uid, params=params)
    print "requesting", r.url

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def query_by_name(url, params, name):
    """
    This adds an artist name to the query parameters before making an API call
    to the function above.
    """
    params["query"] = "artist:" + name
    return query_site(url, params)


def pretty_print(data, indent=4):
    """
    After we get our output, we can use this function to format it to be more
    readable.
    """
    if type(data) == dict:
        print json.dumps(data, indent=indent, sort_keys=True)
    else:
        print data


def main():
    """
    Below is an example investigation to help you get started in your
    exploration. Modify the function calls and indexing below to answer the
    questions on the next quiz.

    HINT: Note how the output we get from the site is a multi-level JSON
    document, so try making print statements to step through the structure one
    level at a time or copy the output to a separate output file. Experimenting
    and iteration will be key to understand the structure of the data!
    """

    # Query for information in the database about bands named Nirvana
    results = query_by_name(ARTIST_URL, query_type["simple"], "Nirvana")
    pretty_print(results)

    # Isolate information from the 4th band returned (index 3)
    print "\nARTIST:"
    pretty_print(results["artists"][3])

    # Query for releases from that band using the artist_id
    artist_id = results["artists"][3]["id"]
    artist_data = query_site(ARTIST_URL, query_type["releases"], artist_id)
    releases = artist_data["releases"]

    # Print information about releases from the selected band
    print "\nONE RELEASE:"
    pretty_print(releases[0], indent=2)

    release_titles = [r["title"] for r in releases]
    print "\nALL TITLES:"
    for t in release_titles:
        print t

if __name__ == '__main__':
    main()

http://musicbrainz.org/ws/2/artist/
{'query': 'artist:Nirvana'}
requesting http://musicbrainz.org/ws/2/artist/?query=artist%3ANirvana&fmt=json
{
    "artists": [
        {
            "area": {
                "id": "6a264f94-6ff1-30b1-9a81-41f7bfabd616", 
                "name": "Finland", 
                "sort-name": "Finland"
            }, 
            "country": "FI", 
            "disambiguation": "Early 1980's Finnish punk band", 
            "id": "85af0709-95db-4fbc-801a-120e9f4766d0", 
            "life-span": {
                "ended": null
            }, 
            "name": "Nirvana", 
            "score": "100", 
            "sort-name": "Nirvana", 
            "tags": [
                {
                    "count": 1, 
                    "name": "punk"
                }, 
                {
                    "count": 1, 
                    "name": "finland"
                }
            ], 
            "type": "Group"
        }, 
        {
            "disambiguatio

requesting http://musicbrainz.org/ws/2/artist/9282c8b4-ca0b-4c6b-b7e3-4f7762dfc4d6?fmt=json&inc=releases

ONE RELEASE:
{
  "barcode": null, 
  "country": "GB", 
  "date": "1969", 
  "disambiguation": "", 
  "id": "0b44cb36-550a-491d-bfd9-8751271f9de7", 
  "packaging": null, 
  "packaging-id": null, 
  "quality": "normal", 
  "release-events": [
    {
      "area": {
        "disambiguation": "", 
        "id": "8a754a16-0027-3a29-b6d7-2b40ea0481ed", 
        "iso-3166-1-codes": [
          "GB"
        ], 
        "name": "United Kingdom", 
        "sort-name": "United Kingdom"
      }, 
      "date": "1969"
    }
  ], 
  "status": "Official", 
  "status-id": "4e304316-386d-3409-af2e-78857eec5cfe", 
  "text-representation": {
    "language": "eng", 
    "script": "Latn"
  }, 
  "title": "To Markos III"
}

ALL TITLES:
To Markos III
Travelling on a Cloud
Songs Of Love And Praise
Songs of Love and Praise
Songs of Love and Praise
All of Us
Secret Theatre
The Story of Simon Simopath
Me And 

### 16. Quiz: Explore JSON

# Lesson 2: Problem set: Data Extraction

### 1. Quiz: using csv module

useful tips: 
* 1. csv.reader(file) --> list
* 2. file.DictReader() --> Dictionary

In [220]:
import csv
import os

DATADIR = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础"
DATAFILE = "745090.csv"


def parse_file(datafile):
    name = ""
    data = []
    with open(datafile,'rb') as f:
        csv_file = csv.reader(f)
        
        for i,value in enumerate(csv_file):
            if i == 0:
                name = value[0].split(',')[1]
            if i not in (0,1):
                data.append(value)
        
    
    # Do not change the line below
    return (name, data)


def test():
    datafile = os.path.join(DATADIR, DATAFILE)
    name, data = parse_file(datafile)

    assert name == "MOUNTAIN VIEW MOFFETT FLD NAS"
    assert data[0][1] == "1:00"
    assert data[2][0] == "01/01/2005"
    assert data[2][5] == "2"


if __name__ == "__main__":
    test()

### 2. Quiz: Excel to CSV

note:

* w = csv.writer(file,mode)
* w.writerow(content)

In [276]:
# -*- coding: utf-8 -*-
import xlrd
import os
import csv
from zipfile import ZipFile
import pprint

DATADIR = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/"
filename = "2013_ERCOT_Hourly_Load_Data.xls"
datafile = os.path.join(DATADIR,filename)
outf = "2013_Max_Loads.csv"
outfile = os.path.join(DATADIR,outf)


def parse_file(datafile):
    workbook = xlrd.open_workbook(datafile)
    sheet = workbook.sheet_by_index(0)
    
    data = {}
    
    for col_i in range(1,sheet.ncols - 1):
        cv = sheet.col_values(col_i,start_rowx=1,end_rowx=None)
        
        maxval = max(cv)
        maxpos = cv.index(maxval) + 1
        maxtime = sheet.cell_value(maxpos,0)
        max_real_time = xlrd.xldate_as_tuple(maxtime,0)
        
        data[sheet.cell_value(0,col_i)] = {"maxval": maxval,
                                           "maxtime": max_real_time
                                          }
    
    return data

def save_file(data, filename):
    
    with open(filename,'w') as f:
        w = csv.writer(f, delimiter='|')
        w.writerow(["Station", "Year", "Month", "Day", "Hour", "Max Load"])

        for k,v in data.iteritems():
            year, month, day, hour, _ , _= v["maxtime"]
            w.writerow([k,year, month, day, hour,v["maxval"]]) 
    
    
def test():
    data = parse_file(datafile)
    save_file(data, outfile)

    number_of_rows = 0
    stations = []

    ans = {'FAR_WEST': {'Max Load': '2281.2722140000024',
                        'Year': '2013',
                        'Month': '6',
                        'Day': '26',
                        'Hour': '17'}}
    correct_stations = ['COAST', 'EAST', 'FAR_WEST', 'NORTH',
                        'NORTH_C', 'SOUTHERN', 'SOUTH_C', 'WEST']
    fields = ['Year', 'Month', 'Day', 'Hour', 'Max Load']

    with open(outfile) as of:
        csvfile = csv.DictReader(of, delimiter="|")
        for line in csvfile:
            station = line['Station']
            if station == 'FAR_WEST':
                for field in fields:
                    # Check if 'Max Load' is within .1 of answer
                    if field == 'Max Load':
                        max_answer = round(float(ans[station][field]), 1)
                        max_line = round(float(line[field]), 1)
                        assert max_answer == max_line

                    # Otherwise check for equality
                    else:
                        assert ans[station][field] == line[field]

            number_of_rows += 1
            stations.append(station)

        # Output should be 8 lines not including header

        assert number_of_rows == 8

        # Check Station Names
        assert set(stations) == set(correct_stations)

        
if __name__ == "__main__":
    test()

### 3. Wrangling Json

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
This exercise shows some important concepts that you should be aware about:
- using codecs module to write unicode files
- using authentication with web APIs
- using offset when accessing web APIs

To run this code locally you have to register at the NYTimes developer site 
and get your own API key. You will be able to complete this exercise in our UI
without doing so, as we have provided a sample result. (See the file 
'popular-viewed-1.json' from the tabs above.)

Your task is to modify the article_overview() function to process the saved
file that represents the most popular articles (by view count) from the last
day, and return a tuple of variables containing the following data:
- labels: list of dictionaries, where the keys are the "section" values and
  values are the "title" values for each of the retrieved articles.
- urls: list of URLs for all 'media' entries with "format": "Standard Thumbnail"

All your changes should be in the article_overview() function. See the test() 
function for examples of the elements of the output lists.
The rest of functions are provided for your convenience, if you want to access
the API by yourself.
"""
import json
import codecs
import requests

URL_MAIN = "http://api.nytimes.com/svc/"
URL_POPULAR = URL_MAIN + "mostpopular/v2/"
API_KEY = { "popular": "",
            "article": ""}


def get_from_file(kind, period):
    filename = "popular-{0}-{1}.json".format(kind, period)
    with open(filename, "r") as f:
        return json.loads(f.read())


def article_overview(kind, period):
    data = get_from_file(kind, period)
    titles = []
    urls =[]

    for article in data:
        section = article["section"]
        title = article["title"]
        titles.append({section: title})
        if "media" in article:
            for m in article["media"]:
                for mm in m["media-metadata"]:
                    if mm["format"] == "Standard Thumbnail":
                        urls.append(mm["url"])

    return (titles, urls)


def query_site(url, target, offset):
    # This will set up the query with the API key and offset
    # Web services often use offset paramter to return data in small chunks
    # NYTimes returns 20 articles per request, if you want the next 20
    # You have to provide the offset parameter
    if API_KEY["popular"] == "" or API_KEY["article"] == "":
        print "You need to register for NYTimes Developer account to run this program."
        print "See Intructor notes for information"
        return False
    params = {"api-key": API_KEY[target], "offset": offset}
    r = requests.get(url, params = params)

    if r.status_code == requests.codes.ok:
        return r.json()
    else:
        r.raise_for_status()


def get_popular(url, kind, days, section="all-sections", offset=0):
    # This function will construct the query according to the requirements of the site
    # and return the data, or print an error message if called incorrectly
    if days not in [1,7,30]:
        print "Time period can be 1,7, 30 days only"
        return False
    if kind not in ["viewed", "shared", "emailed"]:
        print "kind can be only one of viewed/shared/emailed"
        return False

    url += "most{0}/{1}/{2}.json".format(kind, section, days)
    data = query_site(url, "popular", offset)

    return data


def save_file(kind, period):
    # This will process all results, by calling the API repeatedly with supplied offset value,
    # combine the data and then write all results in a file.
    data = get_popular(URL_POPULAR, "viewed", 1)
    num_results = data["num_results"]
    full_data = []
    with codecs.open("popular-{0}-{1}.json".format(kind, period), encoding='utf-8', mode='w') as v:
        for offset in range(0, num_results, 20):        
            data = get_popular(URL_POPULAR, kind, period, offset=offset)
            full_data += data["results"]
        
        v.write(json.dumps(full_data, indent=2))


def test():
    titles, urls = article_overview("viewed", 1)
    assert len(titles) == 20
    assert len(urls) == 30
    assert titles[2] == {'Opinion': 'Professors, We Need You!'}
    assert urls[20] == 'http://graphics8.nytimes.com/images/2014/02/17/sports/ICEDANCE/ICEDANCE-thumbStandard.jpg'


if __name__ == "__main__":
    test()

# Lesson 3: Data in more complex format

### 8. Quiz:  Exracting data

Note:

* find
* findall
* obj.find(tag).text

In [301]:
import xml.etree.ElementTree as ET

article_file = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/exampleResearchArticle.xml"


def get_root(fname):
    tree = ET.parse(fname)
    return tree.getroot()


def get_authors(root):
    authors = []
    for author in root.findall('./fm/bibl/aug/au'):
        data = {
                "fnm": None,
                "snm": None,
                "email": None
        }
        
        data["fnm"] = author.find('fnm').text
        data["snm"] = author.find('snm').text
        data["email"] = author.find('email').text

        authors.append(data)

    return authors


def test():
    solution = [{'fnm': 'Omer', 'snm': 'Mei-Dan', 'email': 'omer@extremegate.com'}, {'fnm': 'Mike', 'snm': 'Carmont', 'email': 'mcarmont@hotmail.com'}, {'fnm': 'Lior', 'snm': 'Laver', 'email': 'laver17@gmail.com'}, {'fnm': 'Meir', 'snm': 'Nyska', 'email': 'nyska@internet-zahav.net'}, {'fnm': 'Hagay', 'snm': 'Kammar', 'email': 'kammarh@gmail.com'}, {'fnm': 'Gideon', 'snm': 'Mann', 'email': 'gideon.mann.md@gmail.com'}, {'fnm': 'Barnaby', 'snm': 'Clarck', 'email': 'barns.nz@gmail.com'}, {'fnm': 'Eugene', 'snm': 'Kots', 'email': 'eukots@gmail.com'}]
    root = get_root(article_file)
    data = get_authors(root)

    assert data[0] == solution[0]
    assert data[1]["fnm"] == solution[1]["fnm"]


test()

### 9. Quiz: handling attributes

Note:  
for item in insr:  
...data["insr"].append(item.attrib["iid"])

In [323]:
import xml.etree.ElementTree as ET

article_file = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/exampleResearchArticle.xml"


def get_root(fname):
    tree = ET.parse(fname)
    return tree.getroot()


def get_authors(root):
    authors = []
    for author in root.findall('./fm/bibl/aug/au'):
        data = {
                "fnm": None,
                "snm": None,
                "email": None,
                "insr": []
        }

        data["fnm"] = author.find('./fnm').text
        data["snm"] = author.find('./snm').text
        data["email"] = author.find('./email').text
        
        insr = author.findall('./insr')
        
        for item in insr:
            data["insr"].append(item.attrib["iid"])
        
        authors.append(data)
        
    return authors


def test():
    solution = [{'insr': ['I1'], 'fnm': 'Omer', 'snm': 'Mei-Dan', 'email': 'omer@extremegate.com'},
                {'insr': ['I2'], 'fnm': 'Mike', 'snm': 'Carmont', 'email': 'mcarmont@hotmail.com'},
                {'insr': ['I3', 'I4'], 'fnm': 'Lior', 'snm': 'Laver', 'email': 'laver17@gmail.com'},
                {'insr': ['I3'], 'fnm': 'Meir', 'snm': 'Nyska', 'email': 'nyska@internet-zahav.net'},
                {'insr': ['I8'], 'fnm': 'Hagay', 'snm': 'Kammar', 'email': 'kammarh@gmail.com'},
                {'insr': ['I3', 'I5'], 'fnm': 'Gideon', 'snm': 'Mann', 'email': 'gideon.mann.md@gmail.com'},
                {'insr': ['I6'], 'fnm': 'Barnaby', 'snm': 'Clarck', 'email': 'barns.nz@gmail.com'},
                {'insr': ['I7'], 'fnm': 'Eugene', 'snm': 'Kots', 'email': 'eukots@gmail.com'}]

    root = get_root(article_file)
    data = get_authors(root)
    

    assert data[0] == solution[0]
    assert data[1]["insr"] == solution[1]["insr"]


test()

### 从网站抓取数据进行分析

data wrangling procedule
(For this exmple)

- Build List of carrier values
- Build List of airport values
- Make HTTP request to download all data
- Then parse the data files

Beautiful Soup将复杂HTML文档转换成一个复杂的树形结构,每个节点都是Python对象,所有对象可以归纳为4种: Tag , NavigableString , BeautifulSoup , Comment .

Tag：
Tag有很多方法和属性,在 遍历文档树 和 搜索文档树 中有详细解释.现在介绍一下tag中最重要的属性: name和attributes

tag.name
tag.attrs

....

遍历文档树
tag的名字

操作文档树最简单的方法就是告诉它你想获取的tag的name.如果想获取 <head> 标签,只要用 soup.head :

这是个获取tag的小窍门,可以在文档树的tag中多次调用这个方法.下面的代码可以获取<body>标签中的第一个b标签:

soup.body.b

通过点取属性的方式只能获得当前名字的第一个tag:

soup.a

如果想要得到所有的a标签,或是通过名字得到比一个tag更多的内容的时候,就需要用到 Searching the tree 中描述的方法,比如: find_all()

soup.find_all('a')

.contents 和 .children

tag的 .contents 属性可以将tag的子节点以列表的方式输出:

通过tag的 .children 生成器,可以对tag的子节点进行循环:

for child in title_tag.children:
    print(child)
    
.string

如果tag只有一个 NavigableString 类型子节点,那么这个tag可以使用 .string 得到子节点:

title_tag.string
 u'The Dormouse's story'

如果一个tag仅有一个子节点,那么这个tag也可以使用 .string 方法,输出结果与当前唯一子节点的 .string 结果相同:

head_tag.contents
 [<title>The Dormouse's story</title>]

head_tag.string
 u'The Dormouse's story'

父节点

兄节点

...

搜索文档树

Beautiful Soup定义了很多搜索方法,这里着重介绍2个: find() 和 find_all() .其它方法的参数和用法类似,请读者举一反三.

正则表达式

如果传入正则表达式作为参数,Beautiful Soup会通过正则表达式的 match() 来匹配内容.下面例子中找出所有以b开头的标签,这表示<body>和b标签都应该被找到:

import re
for tag in soup.find_all(re.compile("^b")):
    print(tag.name)
 body
 b

...

### 19.  Using Beautiful Soup

In [344]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# Please note that the function 'make_request' is provided for your reference only.
# You will not be able to to actually use it from within the Udacity web UI.
# Your task is to process the HTML using BeautifulSoup, extract the hidden
# form field values for "__EVENTVALIDATION" and "__VIEWSTATE" and set the appropriate
# values in the data dictionary.
# All your changes should be in the 'extract_data' function
from bs4 import BeautifulSoup
import requests
import json

html_page = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/page_source.html"


def extract_data(page):
    data = {"eventvalidation": "",
            "viewstate": ""}
    with open(page, "r") as html:
        soup = BeautifulSoup(html,"lxml")
        ev = soup.find(id="__EVENTVALIDATION")
        
        data["eventvalidation"] = ev["value"]

        vs = soup.find(id="__VIEWSTATE")
        data["viewstate"] = vs["value"]


    return data


def make_request(data):
    eventvalidation = data["eventvalidation"]
    viewstate = data["viewstate"]

    r = requests.post("http://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
                    data={'AirportList': "BOS",
                          'CarrierList': "VX",
                          'Submit': 'Submit',
                          "__EVENTTARGET": "",
                          "__EVENTARGUMENT": "",
                          "__EVENTVALIDATION": eventvalidation,
                          "__VIEWSTATE": viewstate
                    })

    return r.text


def test():
    data = extract_data(html_page)
  
    assert data["eventvalidation"] != ""
    assert data["eventvalidation"].startswith("/wEWjAkCoIj1ng0")
    assert data["viewstate"].startswith("/wEPDwUKLTI")

    
test()

In [4]:

from bs4 import BeautifulSoup
import requests

r = requests.get("http://www.transtats.bts.gov/Data_Elements.aspx?Data=2")

soup = BeautifulSoup(r.text)
viewdate_element = soup.find(id="__VIEWSTATE")
viewstate = viewdate_element["value"]

eventvalidation_element = soup.find(id="__EVENTVALIDATION")
eventvalidation = eventvalidation_element["value"]

r = requests.post("http://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
                    data=(('AirportList', "BOS"),
                          ('CarrierList', "VX"),
                          ('Submit', 'Submit'),
                          ("__EVENTTARGET", ""),
                          ("__EVENTARGUMENT", ""),
                          ("__EVENTVALIDATION", eventvalidation),
                          ("__VIEWSTATE", viewstate
                    )))

f = open("virgin_and_logan_airport.html",'w')
f.write(r.text)

# Lesson 4: Problem set: Data in more complex

### 1. Quiz: Carrier list

Good:   
print("\n".join("{} {}".format(el['value'], el.get_text()) for el in airport))

In [74]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Your task in this exercise is to modify 'extract_carrier()` to get a list of
all airlines. Exclude all of the combination values like "All U.S. Carriers"
from the data that you return. You should return a list of codes for the
carriers.

All your changes should be in the 'extract_carrier()' function. The
'options.html' file in the tab above is a stripped down version of what is
actually on the website, but should provide an example of what you should get
from the full file.

Please note that the function 'make_request()' is provided for your reference
only. You will not be able to to actually use it from within the Udacity web UI.
"""

from bs4 import BeautifulSoup
html_page = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/options.html"


def extract_carriers(page):
    data = []

    with open(page, "r") as html:
        soup = BeautifulSoup(html, "lxml")
        
        airport = soup.find_all("option")
      
        for el in airport:
            if len(el["value"]) == 2 :
                data.append(el["value"])
 

    return data


def make_request(data):
    eventvalidation = data["eventvalidation"]
    viewstate = data["viewstate"]
    airport = data["airport"]
    carrier = data["carrier"]

    r = s.post("https://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
               data = (("__EVENTTARGET", ""),
                       ("__EVENTARGUMENT", ""),
                       ("__VIEWSTATE", viewstate),
                       ("__VIEWSTATEGENERATOR",viewstategenerator),
                       ("__EVENTVALIDATION", eventvalidation),
                       ("CarrierList", carrier),
                       ("AirportList", airport),
                       ("Submit", "Submit")))

    return r.text


def test():
    data = extract_carriers(html_page)
    assert len(data) == 16
    assert "FL" in data
    assert "NK" in data

if __name__ == "__main__":
    test()

In [80]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Your task in this exercise is to modify 'extract_carrier()` to get a list of
all airlines. Exclude all of the combination values like "All U.S. Carriers"
from the data that you return. You should return a list of codes for the
carriers.

All your changes should be in the 'extract_carrier()' function. The
'options.html' file in the tab above is a stripped down version of what is
actually on the website, but should provide an example of what you should get
from the full file.

Please note that the function 'make_request()' is provided for your reference
only. You will not be able to to actually use it from within the Udacity web UI.
"""

from bs4 import BeautifulSoup
html_page = "/Users/Simon/Desktop/ThinkPad410/DBS/Tasks/Study/Udacity/进阶/数据提取基础/options.html"


def extract_airports(page):
    data = []

    with open(page, "r") as html:
        soup = BeautifulSoup(html, "lxml")
        
        airport = soup.find_all("option")
      
        for el in airport:
            if len(el["value"]) == 3 and el["value"] != "All":
                data.append(el["value"])
 

    return data


def make_request(data):
    eventvalidation = data["eventvalidation"]
    viewstate = data["viewstate"]
    airport = data["airport"]
    carrier = data["carrier"]

    r = s.post("https://www.transtats.bts.gov/Data_Elements.aspx?Data=2",
               data = (("__EVENTTARGET", ""),
                       ("__EVENTARGUMENT", ""),
                       ("__VIEWSTATE", viewstate),
                       ("__VIEWSTATEGENERATOR",viewstategenerator),
                       ("__EVENTVALIDATION", eventvalidation),
                       ("CarrierList", carrier),
                       ("AirportList", airport),
                       ("Submit", "Submit")))

    return r.text


def test():
    data = extract_airports(html_page)
    assert len(data) == 15
    assert "ATL" in data
    assert "ABR" in data

if __name__ == "__main__":
    test()

# Lesson 5: Data quality

- 审计有效性 （validity）
- 审计准确性  (accuracy)
- 审计完整性  (completeness)
- 审计一致性  (consistency) - 实际上是相信哪一个数据来源的问题
- 审计统一性  (uniformity)  - 如统一字段中使用相同单位


**BLUEPOINT FOR CLEANING**

- Audit your data
- Create a data cleaning plan
    - Identify causes
    - Define operations
    - Test
- Execute the plan (run script)
- Manually correct

***通常，对以上数据进行迭代，通常是2个以上的迭代，然后得到正确的结果***

**Audit Validity**

e.g.
- Data Type
- Range
- Regular expression
- Cross-field constraints
- Foreign Key constraints
- MANDATORY
- UNIQUE

In [84]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
from collections import defaultdict
import re

osm_file = open("chicagociitymap.osm", "r")

street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    

if __name__ == '__main__':
    audit()

#1425: 1
#1850: 1
#500: 1
#510: 1
#575: 1
2105: 1
400: 1
60637: 1
Access: 1
Ave: 2
Avenu: 1
Avenue: 66638
Avenue,: 1
Blvd: 2
Bouevard: 1
Boulevard: 4757
Clinton: 1
Court: 500
Dr: 1
Drive: 733
Lane: 10
LaSalle: 1
level): 1
Market: 52
Monroe: 1
NW: 1
Park: 42
Parkway: 602
Place: 9343
Plaza: 23
Road: 1999
South: 2
St: 10
St.: 2
Street: 47532
Terrace: 50
Walk: 9
Way: 2
Whipple: 1


### 12. Quiz: Correcting validity

Note the find() function   
if row['URI'].find("dbpedia.org") < 0:  
                continue

In [57]:
"""
Your task is to check the "productionStartYear" of the DBPedia autos datafile for valid values.
The following things should be done:
- check if the field "productionStartYear" contains a year
- check if the year is in range 1886-2014
- convert the value of the field to be just a year (not full datetime)
- the rest of the fields and values should stay the same
- if the value of the field is a valid year in the range as described above,
  write that line to the output_good file
- if the value of the field is not a valid year as described above, 
  write that line to the output_bad file
- discard rows (neither write to good nor bad) if the URI is not from dbpedia.org
- you should use the provided way of reading and writing data (DictReader and DictWriter)
  They will take care of dealing with the header.

You can write helper functions for checking the data and writing the files, but we will call only the 
'process_file' with 3 arguments (inputfile, output_good, output_bad).
"""
import csv
import pprint
import re

INPUT_FILE = 'autos.csv'
OUTPUT_GOOD = 'autos-valid.csv'
OUTPUT_BAD = 'FIXME-autos.csv'

def process_file(input_file, output_good, output_bad):

    with open(input_file, "r") as f:
        reader = csv.DictReader(f)
        header = reader.fieldnames
        
        good_data = []
        bad_data = []
        
        low_year = '1886'
        high_year = '2014'
        
        def is_valid_year(year):
            return low_year < year < high_year and year != "NULL"
            
        for i, line in enumerate(reader):                       
            if i > 2:
                productionEndYear = line["productionEndYear"].split("-")[0] 
                uri = line["URI"].split("/")[2]
                
                if is_valid_year(productionEndYear) and uri == "dbpedia.org":
                    good_data.append(line)    
                else:
                    bad_data.append(line)
                    

    with open(output_good, "w") as g:
        writer = csv.DictWriter(g, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in good_data:
            writer.writerow(row)
            
            
    with open(output_bad, "w") as b:
        writer = csv.DictWriter(b, delimiter=",", fieldnames= header)
        writer.writeheader()
        for row in bad_data:
            writer.writerow(row)


def test():

    process_file(INPUT_FILE, OUTPUT_GOOD, OUTPUT_BAD)


if __name__ == "__main__":
    test()
    

# Lesson 6: Problem set: Data quality

### 1. Quiz: Auditing Data Quality

In [101]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up. In the first exercise we want you to audit
the datatypes that can be found in some particular fields in the dataset.
The possible types of values can be:
- NoneType if the value is a string "NULL" or an empty string ""
- list, if the value starts with "{"
- int, if the value can be cast to int
- float, if the value can be cast to float, but CANNOT be cast to int.
   For example, '3.23e+07' should be considered a float because it can be cast
   as float but int('3.23e+07') will throw a ValueError
- 'str', for all other values

The audit_file function should return a dictionary containing fieldnames and a 
SET of the types that can be found in the field. e.g.
{"field1": set([type(float()), type(int()), type(str())]),
 "field2": set([type(str())]),
  ....
}
The type() function returns a type object describing the argument given to the 
function. You can also use examples of objects to create type objects, e.g.
type(1.1) for a float: see the test function below for examples.

Note that the first three rows (after the header row) in the cities.csv file
are not actual data points. The contents of these rows should note be included
when processing data types. Be sure to include functionality in your code to
skip over or detect these rows.
"""
import codecs
import csv
import json
import pprint
from collections import defaultdict

CITIES = 'cities.csv'

FIELDS = ["name", "timeZone_label", "utcOffset", "homepage", "governmentType_label",
          "isPartOf_label", "areaCode", "populationTotal", "elevation",
          "maximumElevation", "minimumElevation", "populationDensity",
          "wgs84_pos#lat", "wgs84_pos#long", "areaLand", "areaMetro", "areaUrban"]

def audit_file(filename, fields):
    fieldtypes = {}
    list_type = []
    
    with open(filename,'r') as f:
        cv = csv.DictReader(f)
        
        for i,value in enumerate(cv):
            if 1 <= i <= 2:
                continue
            
         
            for field in fields:
                tp = type(value[field])
                
                if field in fieldtypes:
                    if tp not in fieldtypes[field]:
                        fieldtypes[field].append(tp)
                else:
                    fieldtypes[field] = list()
                    fieldtypes[field].append(tp)
        

    return fieldtypes


def test():
    fieldtypes = audit_file(CITIES, FIELDS)

    pprint.pprint(fieldtypes)

    #assert fieldtypes["areaLand"] == set([type(1.1), type([]), type(None)])
    #assert fieldtypes['areaMetro'] == set([type(1.1), type(None)])
    
if __name__ == "__main__":
    test()


{'areaCode': [<type 'str'>],
 'areaLand': [<type 'str'>],
 'areaMetro': [<type 'str'>],
 'areaUrban': [<type 'str'>],
 'elevation': [<type 'str'>],
 'governmentType_label': [<type 'str'>],
 'homepage': [<type 'str'>],
 'isPartOf_label': [<type 'str'>],
 'maximumElevation': [<type 'str'>],
 'minimumElevation': [<type 'str'>],
 'name': [<type 'str'>],
 'populationDensity': [<type 'str'>],
 'populationTotal': [<type 'str'>],
 'timeZone_label': [<type 'str'>],
 'utcOffset': [<type 'str'>],
 'wgs84_pos#lat': [<type 'str'>],
 'wgs84_pos#long': [<type 'str'>]}


### 3. Quiz: Fix the area

Note:

reader.next()

SIGNIFICANT DIGIT  - 有效数字

operator

In [127]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
In this problem set you work with cities infobox data, audit it, come up with a
cleaning idea and then clean it up.

Since in the previous quiz you made a decision on which value to keep for the
"areaLand" field, you now know what has to be done.

Finish the function fix_area(). It will receive a string as an input, and it
has to return a float representing the value of the area or None.
You have to change the function fix_area. You can use extra functions if you
like, but changes to process_file will not be taken into account.
The rest of the code is just an example on how this function can be used.
"""
import codecs
import csv
import json
import pprint
import operator

CITIES = 'cities.csv'


def fix_area(area):
    
    if area == "NULL":
        area = None
    else:
        list_area = area.split("|")
        dict_x = {}
        
        for x in list_area:
            x = x.replace('{','')
            x = x.replace('}','')
            n = x.replace('.','')
            
            num = float(x)            
            count = len(n.strip("0"))
                
            dict_x[num] = count 
        
            area = max(dict_x.iteritems(), key=operator.itemgetter(1))[0]
                
    return area



def process_file(filename):
    # CHANGES TO THIS FUNCTION WILL BE IGNORED WHEN YOU SUBMIT THE EXERCISE
    data = []

    with open(filename, "r") as f:
        reader = csv.DictReader(f)

        #skipping the extra metadata
        for i in range(3):
            l = reader.next()

        # processing file
        for line in reader:
            # calling your function to fix the area value
            if "areaLand" in line:
                line["areaLand"] = fix_area(line["areaLand"])
            data.append(line)

    return data


def test():
    data = process_file(CITIES)
    print data[8]["areaLand"]
    print "Printing three example results:"
    for n in range(880,900):
        pprint.pprint(data[n]["areaLand"])  


if __name__ == "__main__":
    test()

None
Printing three example results:
350166000.0
41698800.0
21496900.0
6733970.0
23050900.0
13985900.0
25122900.0
28489900.0
52576800.0
20978900.0
23309900.0
30561900.0
80289600.0
224293000.0
34187800.0
23309900.0
101787000.0
31597900.0
55166700.0
63713700.0
